# Categorical Feature Encoding Classification

### *Bu proje, kategorik verilerin makine öğrenimi modellerinde nasıl işleneceği ve kodlanacağı üzerine bir uygulamadır. Kategorik veriler, sayısal olmayan ve genellikle metinsel değerler içeren verilerdir. Bu tür veriler, özellikle doğru bir şekilde işlendiğinde, makine öğrenimi modellerinin performansını önemli ölçüde etkileyebilir. Projenin amacı, bu verileri en iyi şekilde kullanarak doğru tahminler yapabilen bir model geliştirmektir.*

Tabloya baktığımızda, veri setindeki sütunların çeşitli kategorik özellikler içerdiğini görebiliriz:

* İkili (Binary) Özellikler (bin_0, bin_1, bin_2, bin_3, bin_4): Bu sütunlar, muhtemelen evet/hayır, var/yok gibi iki kategori içeren özelliklerdir. Örneğin, bin_3 ve bin_4 sütunları 'T' ve 'Y' gibi değerler içeriyor.

* Nominal Özellikler (nom_0 - nom_9): Bu sütunlar, belirli bir sıralama olmaksızın çeşitli kategorileri temsil ediyor. Örneğin, nom_0 sütununda 'Green', 'Blue', 'Red' gibi renkler bulunuyor.

* Ordinal Özellikler (ord_0 - ord_5): Bu sütunlar, bir sıralama veya derecelendirme içeriyor. Örneğin, ord_1 sütununda 'Grandmaster', 'Expert' gibi dereceler bulunuyor.

* Döngüsel (Cyclical) Özellikler (day, month): Bu sütunlar, günler ve aylar gibi zamanla ilgili döngüsel verileri temsil ediyor.

* Hedef Sütun (target): Bu, tahmin etmeye çalıştığınız hedef değişkendir.

Bu veri seti üzerinde yapmamız gereken temel görev, bu kategorik özellikleri uygun şekilde kodlamak ve bir makine öğrenimi modeli kullanarak target sütununu tahmin etmektir.

In [28]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [3]:
train = pd.read_csv('/kaggle/input/cat-in-the-dat/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/cat-in-the-dat/test.csv', index_col='id')
submission = pd.read_csv('/kaggle/input/cat-in-the-dat/sample_submission.csv', index_col='id')

In [4]:
train.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


In [5]:
test.head()

,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
id,,,,,,,,,,,,,,,,,,,,,
300000,0,0,1,T,Y,Blue,Triangle,Axolotl,Finland,Piano,...,9d117320c,3c49b42b8,2,Novice,Warm,j,P,be,5,11
300001,0,0,0,T,N,Red,Square,Lion,Canada,Piano,...,46ae3059c,285771075,1,Master,Lava Hot,l,A,RP,7,5
300002,1,0,1,F,Y,Blue,Square,Dog,China,Piano,...,b759e21f0,6f323c53f,2,Expert,Freezing,a,G,tP,1,12
300003,0,0,1,T,Y,Red,Star,Cat,China,Piano,...,0b6ec68ff,b5de3dcc4,1,Contributor,Lava Hot,b,Q,ke,2,3
300004,0,1,1,F,N,Red,Trapezoid,Dog,China,Piano,...,f91f3b1ee,967cfa9c9,3,Grandmaster,Lava Hot,l,W,qK,4,11


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300000 entries, 0 to 299999
Data columns (total 24 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   bin_0   300000 non-null  int64 
 1   bin_1   300000 non-null  int64 
 2   bin_2   300000 non-null  int64 
 3   bin_3   300000 non-null  object
 4   bin_4   300000 non-null  object
 5   nom_0   300000 non-null  object
 6   nom_1   300000 non-null  object
 7   nom_2   300000 non-null  object
 8   nom_3   300000 non-null  object
 9   nom_4   300000 non-null  object
 10  nom_5   300000 non-null  object
 11  nom_6   300000 non-null  object
 12  nom_7   300000 non-null  object
 13  nom_8   300000 non-null  object
 14  nom_9   300000 non-null  object
 15  ord_0   300000 non-null  int64 
 16  ord_1   300000 non-null  object
 17  ord_2   300000 non-null  object
 18  ord_3   300000 non-null  object
 19  ord_4   300000 non-null  object
 20  ord_5   300000 non-null  object
 21  day     300000 non-null  int64 
 22  m

In [29]:
# Ayırma özelliği ve hedef sütunu
X = train.drop(columns=['target', 'bin_0'])
y = train['target']

# Özelliklerin isimlerini ayırma
ordinal_features = [f'ord_{i}' for i in range(6)]
nominal_features = [col for col in X.columns if col not in ordinal_features]

# Ön işleme adımları
preprocessor = ColumnTransformer(
    transformers=[
        ('ord', OrdinalEncoder(), ordinal_features),
        ('nom', OneHotEncoder(handle_unknown='ignore'), nominal_features)
    ])

# Karar Ağacı modeli
model = Pipeline(steps=[('preprocessor', preprocessor),
                            ('classifier', DecisionTreeClassifier())])

# Modelin eğitilmesi
model.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('ord', OrdinalEncoder(),
                                                  ['ord_0', 'ord_1', 'ord_2',
                                                   'ord_3', 'ord_4', 'ord_5']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['bin_1', 'bin_2', 'bin_3',
                                                   'bin_4', 'nom_0', 'nom_1',
                                                   'nom_2', 'nom_3', 'nom_4',
                                                   'nom_5', 'nom_6', 'nom_7',
                                                   'nom_8', 'nom_9', 'day',
                                                   'month'])])),
                ('classifier', DecisionTreeClassifier())])

In [30]:
# Test verilerini hazırlama ve tahmin yapma
X_test = test.drop(columns=['bin_0'])
predictions = model.predict_proba(X_test)
predictions

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

In [33]:
# Pozitif sınıfın olasılıklarını al (model.predict_proba ile elde edilen ikinci sütun)
submission_probabilities = predictions[:, 1]

In [43]:
submission["target"] = submission_probabilities

In [44]:
submission.to_csv("submission.csv")